In [ ]:
!pip install datasets torch==1.13.1 transformers==4.30.1 accelerate==0.21.0 bitsandbytes peft trl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    

In [ ]:
from google.colab import files

uploaded = files.upload()

# here the dataset that we are taking needs to be in JSONL format.

Saving copywrite_ft.jsonl to copywrite_ft.jsonl


In [ ]:
# pip install datasets


In [ ]:

from datasets import load_dataset

# Load your JSON dataset from Gdrive
dataset = load_dataset('json',
                        data_files="/content/copywrite_ft.jsonl",
                        split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
# we are using AutoModelForCausalLM to specify the model, AutoTokenizer to tokenize and to quantize we are using BitsAndBytesConfig

In [ ]:

from huggingface_hub import login
login()

AutoTokenizer is a class from the Transformers library that is used to load and use tokenizers for Transformers models. Tokenizers are used to split text into tokens, which are the basic units of input for Transformers models.

Output Dictionary: {'input_ids': [25005, 2002, 2881, 2001, 6028], 'token_type_ids': [0, 0, 0, 0, 0]}

Quantization of Transformers models using the BitsAndBytes library

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)


tokenizer.pad_token = tokenizer.eos_token



In [ ]:

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # Quantize to 4-bit precision

    bnb_4bit_quant_type="nf4", #  NF4 quantization type should be used.
    bnb_4bit_compute_dtype=torch.float16, # Model should be computed in float16 precision for better accuracy
)

AutoModelForCausalLM is a class from the Transformers library that is used to load and use causal language models (CLMs). CLMs are a type of language model that can generate text by predicting the next word in a sequence, given the previous words in the sequence.

Used to fine-tune pre-trained CLMs on custom datasets

In [ ]:
# Performing quantisation on the model
model = AutoModelForCausalLM.from_pretrained(
    model_name, # Load this model
    quantization_config=bnb_config, # Defined this variable above specifying how the quantization should be done. Quantize like this
    trust_remote_code=True # download the model from the Hugging Face Hub if it is not already cached locally
)

# Disable the caching mechanism of the Transformers model.
# BitsAndBytes library does not support caching for quantized models.
model.config.use_cache = False # to reduce the memory usage, we can set the cache to false.

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from peft import LoraConfig, get_peft_model


''' Strength of the LoRA adaptation.
 Higher values of lora_alpha will result in stronger adaptation,
 but may also lead to overfitting.'''
lora_alpha = 16

lora_dropout = 0.1 # drop out rate in the layers.


lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)



In [ ]:
# Create PeftModel
# Base model - which you can load from the 🤗 Transformers library
# PeftConfig containing the instructions for how to configure a model for LoRa method.
model = get_peft_model(model,peft_config)

In [ ]:
model.print_trainable_parameters()

trainable params: 33,554,432 || all params: 6,771,970,048 || trainable%: 0.49548996469513035


In [ ]:
# Setting all the hyperparameters required for the training
from transformers import TrainingArguments

output_dir = "/content/drive/MyDrive/training data/results" # here the result would be stored

# Number of training examples per device in each batch
per_device_train_batch_size = 4

# Number of batches to accumulate gradients before performing an update to the model weights
gradient_accumulation_steps = 4

# Specialized optimizer for training LLMs on GPUs
optim = "paged_adamw_32bit"

# How often to save the model checkpoint during training
save_steps = 100

# How often to log the training metrics during training
logging_steps = 10

learning_rate = 2e-4
max_grad_norm = 0.3

# Epoch
max_steps = 100

# How quickly the learning rate is increased during the initial stages of training
warmup_ratio = 0.03

# Keeps the learning rate constant throughout training
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:

from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model, # peft model
    train_dataset=dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    args=training_arguments,
)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Convert all of the normalization layers in a Transformers model to float32 precision. This can be helpful for improving the performance of the model on GPUs

In [ ]:

for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

  # Converting all the normalization layer in the transformer model to a float32 precision. This helps us to improve the performance of the model when we use a GPU

In [ ]:


trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.358200
20,1.293100
30,0.689700
40,0.304800
50,0.122500
60,0.098700
70,0.091700
80,0.082500
90,0.073400
100,0.071500


TrainOutput(global_step=100, training_loss=0.5186059576272964, metrics={'train_runtime': 279.0026, 'train_samples_per_second': 5.735, 'train_steps_per_second': 0.358, 'total_flos': 1645885204561920.0, 'train_loss': 0.5186059576272964, 'epoch': 50.0})

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


The trainer.model.module attribute returns the underlying model that is being trained by the Trainer class. If the Trainer class is being used for distributed or parallel training, then the trainer.model.module attribute will be a distributed or parallel model. In this case, it is important to save the trainer.model.module attribute instead of the trainer.model attribute.

The hattr function is used to check if the trainer.model attribute has a module attribute. If it does, then the trainer.model.module attribute is saved. Otherwise, the trainer.model attribute is saved.

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("/content/drive/MyDrive/training data/outputs")
# here we are loading the pretrained lora model


In [ ]:
lora_config = LoraConfig.from_pretrained('/content/drive/MyDrive/training data/outputs')
model = get_peft_model(model, lora_config)


In [ ]:
# For inferencing we are giving the human input and we specified the output as well.
text = "###Human: Give me some good reasons to buy the copyrighter web app. ### Assistant: "
device = "cuda:0"

# Tokenize a text input and convert it to a PyTorch tensor on the specified device.
inputs = tokenizer(text, return_tensors="pt").to(device) # tokenizing the input onto the specified device

In [ ]:
model.print_trainable_parameters()

# here we can see that the trainable parameters are now 0 as we have already trained the model.

trainable params: 0 || all params: 6,771,970,048 || trainable%: 0.0


Generate text using a PyTorch language model.
The model.generate() function is used to generate text from the given input IDs. The max_new_tokens argument specifies the maximum number of tokens to generate.

The tokenizer.decode() function is used to decode the generated token IDs into text. The skip_special_tokens=True argument specifies that the tokenizer should skip the special tokens (such as the start-of-sequence and end-of-sequence tokens) when decoding the text.

In [ ]:
# Here we are using generate function to generate text from the input ids.
# we will get them as ids and we will then convert and decode them into text for which we will use tokenizer.decode
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

###Human: Give me some good reasons to buy the copyrighter web app. ### Assistant:  Certainly! Here are some good reasons to consider purchasing the Copyright web app:

1. Time-saving: The Copyright web app is designed to streamline the copyright registration process, saving you time and effort. With just a few clicks, you can register your work and receive a legally binding copyright certificate.

2. Accurate registration: The app uses advanced algorithms to ensure that your work is registered accurately and efficiently. This reduces the


In [ ]:

outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

###Human: Give me some good reasons to buy the copyrighter web app. ### Assistant: 

Certainly! Here are some good reasons to consider purchasing the Copyright Web App:

1. Time-saving: The app automates the copyright registration process, saving you time and effort.
2. Accuracy: The app ensures that your copyright registration is accurate and complete, reducing the risk of errors or omissions.
3. Ease of use: The app is user-friendly and easy to use, making it accessible to
